## Introduction

In this notebook, We will be creating a model where we will ensemble the results of three regressors namely Random Forest, Xgboost and LGBM 😎. I have not done any Exploratory data analysis and have included only the very basic feature engineering. However, I have plans to improve on that to see if the performance and can be improved or not. Hope the notebook helps you. Thank you and a upvote would mean a lot. 😉

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics 
from lightgbm import LGBMRegressor 
        
input_path = Path('/kaggle/input/tabular-playground-series-feb-2021/')

# Read in the data files

In [ ]:
train = pd.read_csv(input_path / 'train.csv', index_col='id')
display(train.head())

In [ ]:
test = pd.read_csv(input_path / 'test.csv', index_col='id')
display(test.head())

In [ ]:
submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')
display(submission.head())

## Feature Engineering 

#### So lets use label encoding because I will be making mostly tree based models. 

For reference, when to go for label encoding and OHE ( One Hot Encoding ) - 
1. Basically tree based models know how to handle categorical variables so label encoding is preferred over categorical. For non tree based models genrally OHE is better. 
2. If the categorical values are not ordinal then OHE should be preferred over label encoding. 
3. OHE has a problem that it gives rises to the curse of dimensionality when the ordanlity is high. So for good result, one needs to do PCA after OHE. 

Some more reasons are there as well but these three are generally my main guideline. 


In [ ]:
for c in train.columns:
    if train[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)
        
display(train.head())

In [ ]:
target = train.pop('target')

Standardizing the data, though Decision Tree and therefore Random Forest do not need standardization. 

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

In [ ]:
from pandas import DataFrame
train = DataFrame(train)
test = DataFrame(test)

Splitting the data into training set and testing set. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, target, train_size=0.90)

## Model Creation

So we will be creating a ensemble model that will combine the results of Xgboost, RandomForest and LightGBM. 

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestRegressor()
rf.fit(X_train,y_train)
y_preds = rf.predict (X_test)
print(mean_squared_error(y_test,y_preds))

In [ ]:
lgbm = LGBMRegressor()
lgbm.fit(X_train,y_train)
y_pred = lgbm.predict(X_test)
mse_l = mean_squared_error(y_test,y_pred)
print(mse_l)

In [ ]:
import xgboost as xg
xgr = xg.XGBRegressor()
xgr.fit(X_train,y_train)
y_preds = xgr.predict(X_test)
print(mean_squared_error(y_test,y_preds))

In [ ]:
from sklearn.ensemble import VotingRegressor
#create a dictionary of our models
estimators=[('RandomForest', rf), ('LightGBM',lgbm), ('xgboost', xgr)]
#create our voting classifier, inputting our models
ensemble = VotingRegressor(estimators)

In [ ]:
ensemble.fit(X_train,y_train)
y_preds = ensemble.predict(X_test)
print(mean_squared_error(y_test,y_preds))

In [ ]:

lgbm.fit(train, target)
submission['target'] = lgbm.predict(test)
submission.to_csv('lgbm.csv')


In [ ]:
submission